
### **Chapter 5.1: Model Predictive Control**



In this chapter, we introduce model predictive control (MPC), a powerful and widely used framework for solving constrained optimal control problems in both linear and nonlinear systems. Unlike ILQR, which computes a complete open-loop policy offline, MPC solves a finite-horizon optimal control problem (OCP) at each time step, using the current system state as the initial condition. Only the first control input of the optimized sequence is applied before the horizon shifts forward and the process repeats — a concept known as receding horizon control.

We begin by motivating the use of MPC in comparison to open-loop OCP formulations, emphasizing its ability to incorporate feedback. The first part of this chapter focuses on linear MPC (LMPC), where we demonstrate how to reformulate the control problem into a quadratic program (QP), implement the controller, and explore how the horizon and the terminal components affect the LMPC performance.

In the second part, we extend this formulation to nonlinear MPC (NMPC) by leveraging state-of-the-art OCP solvers such as Acados, and walk through the key components of an NMPC pipeline: from solver setup to closed-loop control. Finally, we provide a comparative study between NMPC and classical controllers, like ILQR, highlighting MPC's advantage in handling constraints. We conclude by discussing how key hyperparameters, particularly the prediction horizon, affect the performance and computational cost of MPC.

All the contents are summarized in the table below.  


<table border="1" style="border-collapse: collapse; text-align: center;">
  <!-- Title Row -->
  <tr>
    <th colspan="2" style="text-align:center">Content of Chapter 5.1 Exercise</th>
  </tr>

  <!-- Row group 1 -->
  <tr>
    <td rowspan="2">Open-Loop</td>
    <td>Example 1: Cost Designs in Optimal Control</td>
  </tr>
  <tr>
    <td>Example 2: Open-Loop Limitations</td>
  </tr>

  <!-- Row group 2 -->
  <tr>
    <td rowspan="4">Linear MPC</td>
    <td>Task 1: Linear MPC Implementation</td>
  </tr>
  <tr>
    <td>Task 2: Simulation and Visualization</td>
  </tr>
  <tr>
    <td>Task 3: Horizon and Terminal Components</td>
  </tr>
  <tr>
    <td>Task 4: LMPC vs. LQR</td>
  </tr>

  <!-- Row group 3 -->
  <tr>
    <td rowspan="4">Nonlinear MPC</td>
    <td>Task 1: Solver Configuration</td>
  </tr>
  <tr>
    <td>Task 2: Control Loop Definition</td>
  </tr>
  <tr>
    <td>Task 3: Simulation and Visualization</td>
  </tr>
  <tr>
    <td>Task 4: NMPC vs. ILQR</td>
  </tr>

  <!-- Row group 4 -->
  <tr>
    <td rowspan="1">Hyperparameter Selection</td>
    <td>Example: Prediction Horizon</td>
  </tr>

</table>

First, we need to set up our Python environment and import relevant packages.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from rest.utils import *

import numpy as np
import casadi as ca

### **Problem Setup:**

- Task: starting from given initial position $p_0$, reach a given target position $p_T$ (stabilization)

- Slope profile (height $h$ with reference to horizontal displacement $p$):  
   - case 1: zero slope (linear case), $h(p) = c$
   - case 2: constant slope (linear case), $h(p) = \frac{\pi}{18} \cdot p$
   - case 3: varying slope for small disturbances (nonlinear case), $h(p) = k \cdot \cos(18 p)$
   - case 4: varying slope for under actuated case (nonlinear case), $h(p) = \begin{cases} k \cdot \sin(3 p), & p \in [- \frac{\pi}{2}, \frac{\pi}{6}] \\ k, & p \in (-\infty, -\frac{\pi}{2}) \cup (\frac{\pi}{6}, \infty) \end{cases}$

- System dynamics of 1d mountain car model (in state space representation): 
   - state vector $\boldsymbol{x} = [p, v]^T$
   - input vector $u$
   - system dynamics:
   \begin{align*}
     \begin{bmatrix} \dot{p} \\ \dot{v} \end{bmatrix} = \begin{bmatrix} v \\ - g \sin(\theta) \cos(\theta) \end{bmatrix} + \begin{bmatrix} 0 \\ \cos(\theta)  \end{bmatrix} u
   \end{align*}

### **Preparation: Mountain Car Environment and the System Dynamics**

In the previous exercise, we demonstrated how to define a symbolic function using CasADi, including the definition of the mountain profile as a function of $p$, deriving the conversion formulas between the slope profile $h(p)$ and the inclination angle $\theta(p)$, and establishing the system's dynamics. These formulas have already been integrated into the class `Env` and `Dynamics`. In this chapter, we will specify the arguments and instantiate these classes directly to utilize their functionalities.

- Parameters in the task:  

   - case: 1 (linear case)
   
   - initial state: $\boldsymbol{x}_0 = [-0.5, 0.0]^T$

   - target state: $\boldsymbol{x}_T = [0.6, 0.0]^T$

   - input constraints: $ \mathcal{U} \in [-1.0, 1.0]$


In [ ]:
# Define profile of slope, the initial / target state
case = 1 # 1 or 2 or 3 or 4
initial_position = -0.5
initial_velocity = 0.0
target_position = 0.6
target_velocity = 0.0

# Define the constraints for input (potentially used)
input_lbs = -1.0
input_ubs = 1.0

# Case 1: unconstrained case
env = Env(case, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
dynamics = Dynamics(env)

# Case 2: constrained case
env_constr = Env(case, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]),
                 input_lbs=input_lbs, input_ubs=input_ubs)
dynamics_constr = Dynamics(env_constr)

env.test_env() # show shape of slope (left side) and theta curve (right side) 


<br>

----

### **Part (a): Open-Loop**

We first consider a simple constrained open-loop optimal control problem (OCP), which is solved offline over the entire time horizon by formulating and optimizing a quadratic (or more generally, nonlinear) programming problem to obtain the input sequence that minimizes a given objective function:

$$
J = g_N(\boldsymbol{x}_{N}) + \sum_{k=0}^{N-1} g_k(\boldsymbol{x}_{k}, \boldsymbol{u}_k)
$$

subject to:

$$
\boldsymbol{x}_{k+1} = \boldsymbol{A} \boldsymbol{x}_k + \boldsymbol{B} \boldsymbol{u}_k, \quad \forall k \in \{0, \dots, N-1\},
$$

$$
\boldsymbol{x}_k \in \mathcal{X}, \quad \forall k \in \{0, \dots, N\},
$$

$$
\boldsymbol{u}_k \in \mathcal{U}, \quad \forall k \in \{0, \dots, N-1\},
$$

$$
\boldsymbol{x_{0}} = \boldsymbol{\overline{x}_{0}}
$$

By fixing the algorithmic hyperparameters and applying the optimal input sequence obtained from solving the OCP to the simulation environment, we obtain the following simulation results.


In [ ]:
# Define weight matrix in stage and terminal cost and the horizon for MPC 
Q = np.diag([1, 1])
R = np.array([[0.1]])
Qf = Q  # terminal cost (here the same as the stage cost)

# Define parameters of simulation
freq = 20 # controll frequency
t_terminal = 8 # time length of simulation

# Compute horizon for open-loop optimal controller (full horizon)
N = freq * t_terminal # here 160

In [ ]:
# Define the OCP controller
controller_ocp_2n = LinearOCPController(env_constr, dynamics_constr, Q, R, Qf, freq, N, name='OCP-2-norm')

# Instantiate the simulator, run the simulation, and plot the results
simulator_ocp_2n = Simulator(dynamics_constr, controller_ocp_2n, env_constr, 1/freq, t_terminal)
simulator_ocp_2n.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_ocp_2n = Visualizer(simulator_ocp_2n)
visualizer_ocp_2n.display_plots()
visualizer_ocp_2n.display_animation()


<br>

#### **Example 1: Cost Designs in Optimal Control**  

Apart from the standard quadratic cost (2-norm), one can also use different norms, such as the 1-norm or infinity-norm, to achieve different control characteristics. These alternative cost designs lead to different system behavior and are particularly useful when specific structural or robustness properties are desired. 

For example, the following shows a design of an MPC cost function based on the 1-norm:

$$
\begin{aligned}
& \min_{u_0, u_1, \dots, u_{N-1}} && \sum_{k=0}^{N-1} |u_k| \\
& \text{subject to} \quad 
&& x_{k+1} = x_k + u_k, \quad \forall k \in \{0, 1, \dots, N-1\}, \\
&&& x_0 = \bar{x}_0,\\
&&& \boxed{x_N = x_g,} \\
&&& \boldsymbol{x}_k \in \mathcal{X}, \quad \forall k \in \{0, \dots, N\},\\
&&& \boldsymbol{u}_k \in \mathcal{U}, \quad \forall k \in \{0, \dots, N-1\}
\end{aligned}
$$

*Note 1: the cost function here is not directly defined in terms of the state $\boldsymbol{x}$. Therefore, to ensure that the system state remains close to the desired target, we impose a terminal constraint that explicitly forces the final state in each MPC horizon to be equal to the target state.*

*Note 2: recall from the course that we have shown how to reformulate the OCP problem with 1-norm cost into a QP problem. But here there's no need to do such reformulation because the solver we used (Acados, shown in Section 3) could do that internally for us*
<br>

Here, we directly provide an implementation of the controller. You can simply instantiate it and run the simulation to observe the results.

In [ ]:
## 1-norm cost design

# Define the OCP controller
controller_ocp_1n = LinearOCP1NormController(env_constr, dynamics_constr, freq, N)

# Instantiate the simulator, run the simulation, and plot the results
simulator_ocp_1n = Simulator(dynamics_constr, controller_ocp_1n, env_constr, 1/freq, t_terminal)
simulator_ocp_1n.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_ocp_1n = Visualizer(simulator_ocp_1n)
visualizer_ocp_1n.display_plots()
visualizer_ocp_1n.display_animation()

Another popular choice is the infinity norm, as shown in the following form:

$$
\begin{aligned}
& \min_{u_0, u_1, \dots, u_{N-1}} && \max_{k \in \{0, \dots, N-1\}} |u_k| \\
& \text{subject to} \quad 
&& x_{k+1} = x_k + u_k, \quad \forall k \in \{0, 1, \dots, N-1\}, \\
&&& x_0 = \bar{x}_0,\\
&&& x_N = x_g,  \\
&&& \boldsymbol{x}_k \in \mathcal{X}, \quad \forall k \in \{0, \dots, N\},\\
&&& \boldsymbol{u}_k \in \mathcal{U}, \quad \forall k \in \{0, \dots, N-1\}
\end{aligned}
$$

Since most solvers cannot directly handle non-smooth objectives involving the maximum operator (such as $\max_{k} |u_k|$), we reformulate the problem by introducing an auxiliary variable $q$, resulting in an equivalent linear program. This transformation (also known as epigraph reformulation) makes the problem solver-compatible while preserving the original optimization:


$$
\begin{aligned}
& \min_{q, u_0, \dots, u_{N-1}} && \boxed{q} \\
& \text{subject to} \quad 
&& x_{k+1} = x_k + u_k, \quad \forall k \in \{0, \dots, N-1\}, \\
&&& x_0 = \bar{x}_0, \\
&&& x_N = x_g, \\
&&& \boxed{-q \le u_k \le q, \quad \forall k \in \{0, \dots, N-1\},}  \\
&&& \boldsymbol{x}_k \in \mathcal{X}, \quad \forall k \in \{0, \dots, N\},\\
&&& \boldsymbol{u}_k \in \mathcal{U}, \quad \forall k \in \{0, \dots, N-1\}
\end{aligned}
$$

Similar to the 1-norm class, you can simply instantiate the inf-norm class and run the simulation to observe the results.

In [ ]:
## inf-norm cost design
N = freq * t_terminal # here 160

# Define the OCP controller
controller_ocp_infn = LinearOCPInfNormController(env, dynamics, freq, N)

# Instantiate the simulator, run the simulation, and plot the results
simulator_ocp_infn = Simulator(dynamics, controller_ocp_infn, env, 1/freq, t_terminal)
simulator_ocp_infn.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_ocp_infn = Visualizer(simulator_ocp_infn)
visualizer_ocp_infn.display_plots()
visualizer_ocp_infn.display_animation()

#### **Results Analysis**

The following figure illustrates how different norm-based cost functions (specifically, the $l_1$-norm, $l_2$-norm, and $l_{\infty}$-norm) lead to distinct control behaviors, even under the same initial and terminal conditions, dynamics, and input constraints.

From the perspective of optimal control, the choice of the norm directly shapes the optimal control policy:

- The $l_1$-norm penalizes the absolute sum of control inputs, which **promotes sparsity** in the solution. As shown in the input plot, the controller generates long intervals of zero or constant control, applying input only when necessary. This is often desirable when **minimizing actuator usage or energy consumption**.

- The $l_2$-norm, being quadratic and smooth, encourages **smooth and distributed control effort over time**. This leads to continuous acceleration profiles and naturally shaped velocity responses. It is commonly used due to its strong mathematical properties (e.g., uniqueness, differentiability) and **robustness to noise**.

- The $l_{\infty}$-norm minimizes the maximum absolute control effort across the horizon, which often results in **bang-bang-like control**—i.e., switching between the upper and lower bounds. This is advantageous **when enforcing strict limits on peak actuator usage**, but can introduce chattering or less smooth behavior.

This example highlights that the norm selected in cost design is not just a mathematical formality, but will significantly affect the qualitative behavior of the resulting control policy. **The smoothness and sparsity can be tailored by choosing an appropriate norm to reflect the control objective.**

In [ ]:
# Performance comparison under different norm-based cost designs
visualizer_ocp_1n.display_contrast_plots(simulator_ocp_2n, simulator_ocp_infn)

<br>

#### **Example 2: Open-Loop Limitations** 

One advantage of the open-loop algorithm is that it allows for offline pre-computation. However, it also has several limitations:

- The optimization must be performed over the entire simulation horizon. For example, with a simulation duration of $8$ seconds and a control frequency of $20$ Hz, the problem involves $160$ time steps, resulting in a relatively high computational cost;

- Open-loop policies are inherently less robust to noise and model-uncertainty when deployed on a real system. Any disturbance during execution can significantly degrade the performance of the control policy.

In offline computation, we can afford nearly unlimited computational resources and time, so the first drawback is often negligible in practice. However, the second limitation is far more critical: during actual controller deployment, model mismatch and unavoidable disturbances are almost always present.

To illustrate how such uncertainties can degrade the performance of open-loop control, we present a representative example that highlights the vulnerability of open-loop policies under real-world deviations from the nominal model:

&nbsp;&nbsp;&nbsp;&nbsp;1\) Use the dynamics from the **flat case (case 1)** to design the open-loop optimal controller: the discrete dynamics model used in the controller is derived from the cart dynamics on a flat ground, which is the same as `controller_ocp_2n`.

&nbsp;&nbsp;&nbsp;&nbsp;2\) Use the **bumpy case (case 3)** as the simulation environment: the mountain profile is designed as a sinusoidal wave that extends along the horizontal axis. By setting a relatively small amplitude, we aim to simulate minor perturbations commonly encountered in real-world scenarios.

By designing the controller and performing the simulation in different environments, we are able to simulate disturbances that are unknown at the time of controller design, as would be the case in the real world. This allows us to assess the robustness of the controller under model mismatch and unexpected variations. Please run the simulation and observe the results.

In [ ]:
# nonlinear case number
case_disturb = 3

# Instantiate class 'Env'
env_disturb = Env(case_disturb, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
env_disturb.test_env()

# Instantiate class 'Dynamics'
dynamics_disturb = Dynamics(env_disturb)

In [ ]:
# Instantiate the simulator, run the simulation, and plot the results
simulator_ocp_disturb = Simulator(dynamics_disturb, controller_ocp_2n, env_disturb, 1/freq, t_terminal)
simulator_ocp_disturb.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_ocp_disturb = Visualizer(simulator_ocp_disturb)
visualizer_ocp_disturb.display_plots()
visualizer_ocp_disturb.display_animation()

From the simulation results, we can observe that the open-loop controller produces **exactly the same control input sequence as before**. However, due to the presence of persistent small disturbances in the environment, the system behavior deviates from the nominal trajectory. As a result, the controller fails to drive the cart to the target state, highlighting the vulnerability of open-loop control under real-world uncertainties.

So, is there a way to introduce a feedback mechanism into optimal control? Suppose we could :

- Re-solve an open-loop optimal control problem at every time step, using the current state as the new initial condition. $\boldsymbol{x_{k|k}} = \boldsymbol{\overline{x}_{k}}$

- Furthermore, instead of solving the problem over the entire time horizon, we could consider only a shorter prediction horizon into the future $N_{mpc} \ll N_{full}$. This significantly reduces the computational burden, making online optimization feasible.

These ideas form the core of model predictive control (MPC), also known as receding horizon control, which a widely-used approach that solves a constrained optimization problem at each time step. The MPC problem can be formulated as follows:

$$
J_k^*(\boldsymbol{x_k}) = \min_{u_{k|k}, \ldots, u_{k+N-1|k}} 
g_N\left( \boldsymbol{x_{k+N|k}} \right) + \sum_{i=0}^{N-1} g_i\left( \boldsymbol{x_{k+i|k}}, \boldsymbol{u_{k+i|k}} \right)
$$

subject to:

$$
\boldsymbol{x_{k+i+1|k}} = \boldsymbol{A} \boldsymbol{x_{k+i|k}} + \boldsymbol{B} \boldsymbol{u_{k+i|k}}, \quad \forall i \in \{0, \ldots, N-1\},
$$

$$
\boldsymbol{x_{k+i|k}} \in \mathcal{X}, \quad \forall i \in \{0, \ldots, N\},
$$

$$
u_{k+i|k} \in \mathcal{U}, \quad \forall i \in \{0, \ldots, N-1\},
$$

$$
\boldsymbol{x_{k|k}} = \boldsymbol{\overline{x}_{k}}
$$

*Note that: It is important to distinguish the notation used in the OCP formulation from that in the MPC context. In the OCP problem, the index $k$ denotes a discrete time step along the entire simulation horizon. Since the OCP is solved once offline, this global time index is consistent throughout the problem. In contrast, MPC involves solving an optimization problem at each time step $k$ of the simulation. Within each individual MPC problem, a separate index $i$ is introduced to represent the steps along the prediction horizon at time $k$. This separation of $k$ (global time step) and $i$ (local horizon index) helps clarify the distinction between the simulation timeline and the prediction horizon.*

<br>

Based on a simple MPC implementation that we have prepared for you, you can initialize the solver using the same set of hyperparameters as in the previous OCP problem (except for the prediction horizon $N$, which needs to be specified separately), and run simulations to observe the resulting behavior.

In [ ]:
# Specify the prediction horizon (could be much smaller than before)
N=20

# Define the MPC controller
controller_mpc = MPCController(env, dynamics, Q, R, Qf, freq, N, name='MPC_0', verbose=None)

start_time = time.time()

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc_disturb = Simulator(dynamics_disturb, controller_mpc, env_disturb, 1/freq, t_terminal)
simulator_mpc_disturb.run_simulation()

print(f"Average computation time per step: {(time.time()-start_time) / (freq * t_terminal)}")

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc_disturb = Visualizer(simulator_mpc_disturb)
visualizer_mpc_disturb.display_plots()
visualizer_mpc_disturb.display_animation()

#### **Results Analysis**

By comparing the simulation results of open-loop OCP and MPC under environment with uncertainties, we can draw the following conclusions:

- The computation time required for each MPC update is significantly shorter than that of the full OCP (e.g., the average computation time is only $0.1$ ms for $N=20$), making MPC particularly well-suited for computation online.

- By re-solving the optimal control problem at each time step based on the current state (i.e., through online computation), we successfully create a feedback law. 

In summary, MPC provides a close approximation to the OCP solution using a shorter horizon, making it a practical and effective choice. Compared to the open-loop OCP, reducing the prediction horizon in MPC helps decrease the computational load, making it more suitable for online deployment. In turn, online deployment enables the controller to continuously adapt to changes in the system state, account for disturbances or model-mismatch, and incorporate updated sensor measurements or model corrections, thereby improving robustness and responsiveness.


??? offset


<br>

----

### **Part (b): Linear MPC**


In this section, we present the implementation of **model predictive control (MPC)** for linear systems. We begin with a simple case where the system dynamics are assumed to be **linear time-invariant in discrete time** (LTI-DT). This special case is commonly referred to as **linear MPC (LMPC)**.

#### **Problem Definition**

We consider the following objective for LMPC to compute a sequence of control inputs $ \{\boldsymbol{u}_0, \boldsymbol{u}_1, \dots, \boldsymbol{u}_{N-1}\} $ that minimizes the cost:

$$
J = g_N(\boldsymbol{x}_{N}) + \sum_{k=0}^{N-1} g_k(\boldsymbol{x}_{k}, \boldsymbol{u}_k)
$$

subject to:

- the linear discrete-time system dynamics

  $$
  \boldsymbol{x}_{k+1} = \boldsymbol{A} \boldsymbol{x}_k + \boldsymbol{B} \boldsymbol{u}_k, \quad \forall k \in \{0, \dots, N-1\}
  $$

- state and input constraints:

  $$
  \boldsymbol{x}_k \in \mathcal{X}, \quad \forall k \in \{0, \dots, N\},
  $$

  $$
  \boldsymbol{u}_k \in \mathcal{U}, \quad \forall k \in \{0, \dots, N-1\}
  $$

#### **Cost Design**

The cost we used for the following case is designed to be a quadratic cost, which means:

- stage cost $g_k(\boldsymbol{x}_{k}, \boldsymbol{u}_k)$:

$$
g_k(\boldsymbol{x}_{k}, \boldsymbol{u}_k) =  \left( \boldsymbol{x}_k - \boldsymbol{x}_{\text{ref}} \right)^T \boldsymbol{Q} \left( \boldsymbol{x}_k - \boldsymbol{x}_{\text{ref}} \right)
+ \left( \boldsymbol{u}_k - \boldsymbol{u}_{\text{ref}} \right)^T \boldsymbol{R} \left( \boldsymbol{u}_k - \boldsymbol{u}_{\text{ref}} \right)
$$

- terminal cost $g_N(\boldsymbol{x}_{N})$:

$$
g_N(\boldsymbol{x}_{N}) = \left( \boldsymbol{x}_N - \boldsymbol{x}_{\text{ref}} \right)^T \boldsymbol{Q}_f \left( \boldsymbol{x}_N - \boldsymbol{x}_{\text{ref}} \right)
$$



#### **Lifted-Form Formulation**

In the context of MPC, we need to solve a constrained optimization problem over a **finite prediction horizon**. At each time step, we must consider not only the immediate control input, but also how that input will affect future states, costs, and constraints over time.

However, if we naively define separate variables for each time step and impose recursive dynamics $ \boldsymbol{x}_{k+1} = \boldsymbol{A} \boldsymbol{x}_k + \boldsymbol{B} \boldsymbol{u}_k $ explicitly in our optimization code, we end up with a large number of interdependent variables that are hard to manage, especially when using general-purpose numerical solvers.

To systematically express the entire prediction problem as a single standard **quadratic program (QP)**, we introduce the **lifted form**. This formulation involves:

- **Stacking all decision variables** (states and controls) into a single vector $ \boldsymbol{z} $
- **Rewriting dynamics, cost, and constraints** in terms of matrix-vector operations on $ \boldsymbol{z} $

which can be rewritten in the following format:

$$
\min_z \quad \frac{1}{2} \boldsymbol{z}^T \boldsymbol{H} \boldsymbol{z} + \boldsymbol{f}^T \boldsymbol{z}
$$

$$
\text{s.t.} \quad \boldsymbol{A}_{\text{eq}} \boldsymbol{z} = \boldsymbol{b}_{\text{eq}},
$$

$$
 \quad  \quad  \boldsymbol{G} \boldsymbol{z} \leq \boldsymbol{h}
$$

The lifted form offers the following benefits:

- ✅ It transforms the MPC problem into a **structured QP** with a standard form:  
- ✅ It enables the use of **off-the-shelf QP solvers** (e.g. `cvxopt`, `qpoases`, `osqp`) with minimal modification
- ✅ It clarifies the roles of each part of the problem (cost, dynamics, constraints) via matrix structure
- ✅ It easily supports extensions, such as soft constraints, time-varying references, and warm-starting

##### **a) State and Input Variable in Lifted Form**

To formulate the problem as a standard QP, we define a stacked decision variable:

$$
\boldsymbol{z} :=
\begin{bmatrix}
\boldsymbol{x}_0 \\\\ \boldsymbol{x}_1 \\\\ \vdots \\\\ \boldsymbol{x}_N \\\\ \boldsymbol{u}_0 \\\\ \boldsymbol{u}_1 \\\\ \vdots \\\\ \boldsymbol{u}_{N-1}
\end{bmatrix}
\in \mathbb{R}^{(N+1)n_x + N n_u}
$$

##### **b) Quadratic Cost in Lifted Form**

The cost function is written in standard QP form:

$$
\min_z \quad \frac{1}{2} \boldsymbol{z}^T \boldsymbol{H} \boldsymbol{z} + \boldsymbol{f}^T \boldsymbol{z}
$$

where:

- $ \boldsymbol{H} \in \mathbb{R}^{n_z \times n_z} $ is a block-diagonal matrix:

  $$
  \boldsymbol{H} = \text{diag}(
  \underbrace{\boldsymbol{Q}, \dots, \boldsymbol{Q}}_{N-1 \text{ times}}, \boldsymbol{Q}_f, 
  \underbrace{\boldsymbol{R}, \dots, \boldsymbol{R}}_{N \text{ times}}
  )
  $$

- $ f \in \mathbb{R}^{n_z} $ contains the linear terms:

$$
\boldsymbol{f} =
\begin{bmatrix}
- \boldsymbol{Q} \boldsymbol{x}_{\text{ref}} \\\\ \vdots \\\\ - \boldsymbol{Q} \boldsymbol{x}_{\text{ref}} \\\\ - \boldsymbol{Q}_f \boldsymbol{x}_{\text{ref}} \\\\
- \boldsymbol{R} \boldsymbol{u}_{\text{ref}} \\\\ \vdots \\\\ - \boldsymbol{R} \boldsymbol{u}_{\text{ref}}
\end{bmatrix}
$$


##### **c) Equality Constraints (Dynamics) in Lifted Form**

The system dynamics are encoded into equality constraints:

$$
\boldsymbol{A}_{\text{eq}} \boldsymbol{z} = \boldsymbol{b}_{\text{eq}}
$$

Each row of the constraint corresponds to one time step:

- For $ k = 0 $:
  $$
  \boldsymbol{x}_1 = \boldsymbol{A} \boldsymbol{x}_0 + \boldsymbol{B} \boldsymbol{u}_0 \Rightarrow
  \boldsymbol{A}_{\text{eq}}^{(0)} = [-\boldsymbol{A} \; \boldsymbol{I} \; \boldsymbol{0} \dots \boldsymbol{0} \mid -\boldsymbol{B} \; \boldsymbol{0}  \dots \boldsymbol{0}]
  $$

- For $ k = 1, \dots, N-1 $:
  $$
  \boldsymbol{x}_{k+1} = \boldsymbol{A} \boldsymbol{x}_k + \boldsymbol{B} \boldsymbol{u}_k \Rightarrow
  \boldsymbol{A}_{\text{eq}}^{(k)} = [\boldsymbol{0} \dots -\boldsymbol{A} \; \boldsymbol{I} \dots \boldsymbol{0} \mid \boldsymbol{0} \dots -\boldsymbol{B} \dots \boldsymbol{0}]
  $$

- If $ \boldsymbol{x}_0 $ is a variable, we additionally add:
  $$
  \boldsymbol{x}_0 = \boldsymbol{x}_{\text{init}} \Rightarrow \boldsymbol{A}_{\text{eq}}^{\text{init}} = [\boldsymbol{I} \; \boldsymbol{0} \dots \boldsymbol{0} \mid \boldsymbol{0} \dots \boldsymbol{0}]
  $$

Then:

$$
\boldsymbol{A}_{\text{eq}} \in \mathbb{R}^{N n_x + n_x \times n_z}, \quad \boldsymbol{b}_{\text{eq}} = 
\begin{bmatrix}
\boldsymbol{x}_{\text{init}} \\\\ \boldsymbol{0} \\\\ \vdots \\\\ \boldsymbol{0}
\end{bmatrix}
$$


##### **d) Inequality Constraints (State and Input Constraints) in Lifted Form**

Input and state bounds are stacked similarly:

$$
\boldsymbol{G} \boldsymbol{z} \leq \boldsymbol{h}
$$

Where:

- $ \boldsymbol{G} $ includes block entries of $ \pm \boldsymbol{I} $ applied to state and input components of $ \boldsymbol{z} $
- $ \boldsymbol{h} $ contains stacked bounds (e.g., $ \boldsymbol{x}_{\min}, \boldsymbol{x}_{\max}, \boldsymbol{u}_{\min}, \boldsymbol{u}_{\max} $)

<br>

<br>

#### **Task 1: Linear MPC Implementation**

In this step, we implement the core logic of a linear MPC based on a standard **QP** formulation using the **lifted form** and solved via the `cvxopt` solver. This implementation assumes that the system dynamics are linear time-invariant (LTI), and it expresses the full finite-horizon optimal control problem as a single constrained quadratic program.

The implementation follows the classical lifted-form pipeline and includes four key components:

1\) **Linearize the system** and compute the discrete-time matrices $ \boldsymbol{A}_d $, $ \boldsymbol{B}_d $ at the current state;  
   *Hint: use the `get_linearized_AB_discrete()` method from the `Dynamics` class with current state and a nominal control input (typically zero).*

2\) **Construct the cost function** in QP form:  

   The QP cost is written as:  

   $$
   J = \frac{1}{2} \boldsymbol{z}^\top \boldsymbol{H} \boldsymbol{z} + \boldsymbol{f}^\top \boldsymbol{z}
   $$

   where $ \boldsymbol{z} $ is the stacked vector of states and inputs over the horizon:  

   $$
   \boldsymbol{z} = [\boldsymbol{x}_0^\top, \boldsymbol{x}_1^\top, \dots, \boldsymbol{x}_N^\top, \boldsymbol{u}_0^\top, \dots, \boldsymbol{u}_{N-1}^\top]^\top
   $$

   - Matrix `H` is block-diagonal with weights $ \boldsymbol{Q}, \boldsymbol{Q}_f, \boldsymbol{R} $ repeated across the horizon;

   - Vector `f` encodes target tracking;

3\) **Formulate the dynamics as equality constraints**:  

   The dynamics constraints are stacked in lifted form:

   $$
   \boldsymbol{A}_{\text{eq}} \boldsymbol{z} = \boldsymbol{b}_{\text{eq}}
   $$

   - Each row encodes $ \boldsymbol{x}_{k+1} = \boldsymbol{A} \boldsymbol{x}_k + \boldsymbol{B} \boldsymbol{u}_k $

   - The first row optionally enforces $ \boldsymbol{x}_0 = \boldsymbol{x}_{\text{init}} $ as a hard constraint

4\) **Define inequality constraints** for states and controls:  

   These are encoded as:

   $$
   \boldsymbol{G} \boldsymbol{z} \leq \boldsymbol{h}
   $$

   - Each constraint uses $\pm \boldsymbol{I}$ applied to specific blocks of the decision vector `z`

   - You can access lower/upper bounds from `env.state_lbs`, `env.input_ubs`, etc.

5\) **Solve the QP** using the `cvxopt` interface:  

   ```python
                                sol = solvers.qp(matrix(H), matrix(f), matrix(G), matrix(h), matrix(Aeq), matrix(beq))


In [ ]:
from cvxopt import matrix, solvers

class LinearMPCController:
    def __init__(self, 
            env: Env, 
            dynamics: Dynamics, 
            Q: np.ndarray, 
            R: np.ndarray, 
            Qf: np.ndarray, 
            freq: float, 
            N: int, 
            name: str = 'LMPC', 
            type: str = 'MPC', 
            verbose: bool = False
        ) -> None:

        self.env = env
        self.dynamics = dynamics

        self.Q = Q
        self.R = R
        self.Qf = Qf

        self.N = N

        self.name = name
        self.type = type

        self.freq = freq
        self.dt = 1.0 / freq
        
        self.verbose = verbose

        self.dim_states = dynamics.dim_states
        self.dim_inputs = dynamics.dim_inputs

        self.x_pred = None
        self.u_pred = None
    
    @check_input_constraints
    def compute_action(self, current_state, current_time=None):

        x0 = current_state.reshape(-1, 1)
        u0 = np.zeros((self.dim_inputs, 1))

        A_d, B_d = self.dynamics.get_linearized_AB_discrete(x0, u0, self.dt)

        n_vars = self.dim_states * (self.N + 1) + self.dim_inputs * self.N

        # Cost
        H = np.zeros((n_vars, n_vars))
        f = np.zeros((n_vars, 1))

        x_ref = self.env.target_state.reshape(-1, 1)  # assumed constant
        u_ref = np.array(self.dynamics.get_equilibrium_input(x_ref)).reshape(-1, 1)        # assumed constant

        for k in range(self.N):
            idx_x = slice(k * self.dim_states, (k + 1) * self.dim_states)
            idx_u = slice((self.N + 1) * self.dim_states + k * self.dim_inputs,
                          (self.N + 1) * self.dim_states + (k + 1) * self.dim_inputs)

            Qk = self.Q
            H[idx_x, idx_x] = Qk
            H[idx_u, idx_u] = self.R

            f[idx_x] = -Qk @ x_ref
            f[idx_u] = -self.R @ u_ref

        # Terminal cost
        idx_x_terminal = slice(self.N * self.dim_states, (self.N + 1) * self.dim_states)
        H[idx_x_terminal, idx_x_terminal] = self.Qf
        f[idx_x_terminal] = -self.Qf @ x_ref

        # Dynamics constraints
        Aeq = []
        beq = []

        for k in range(self.N):
            row = np.zeros((self.dim_states, n_vars))
            idx_xk = slice(k * self.dim_states, (k + 1) * self.dim_states)
            idx_xk_next = slice((k + 1) * self.dim_states, (k + 2) * self.dim_states)
            idx_uk = slice((self.N + 1) * self.dim_states + k * self.dim_inputs,
                           (self.N + 1) * self.dim_states + (k + 1) * self.dim_inputs)
            row[:, idx_xk_next] = -np.eye(self.dim_states)
            row[:, idx_xk] = A_d
            row[:, idx_uk] = B_d
            Aeq.append(row)
            beq.append(np.zeros((self.dim_states, 1)))

        # Add x0 = current_state as hard equality constraint
        row0 = np.zeros((self.dim_states, n_vars))
        row0[:, :self.dim_states] = np.eye(self.dim_states)
        Aeq.insert(0, row0)
        beq.insert(0, x0)

        Aeq = np.vstack(Aeq)
        beq = np.vstack(beq)

        # Inequality constraints
        G_list = []
        h_list = []

        for k in range(self.N + 1):
            # State constraints
            if self.env.state_lbs is not None:
                Gx_l = np.zeros((self.dim_states, n_vars))
                Gx_l[:, k * self.dim_states:(k + 1) * self.dim_states] = -np.eye(self.dim_states)
                G_list.append(Gx_l)
                h_list.append(-np.array(self.env.state_lbs).reshape(-1, 1))
            if self.env.state_ubs is not None:
                Gx_u = np.zeros((self.dim_states, n_vars))
                Gx_u[:, k * self.dim_states:(k + 1) * self.dim_states] = np.eye(self.dim_states)
                G_list.append(Gx_u)
                h_list.append(np.array(self.env.state_ubs).reshape(-1, 1))

        for k in range(self.N):
            # Input constraints
            if self.env.input_lbs is not None:
                Gu_l = np.zeros((self.dim_inputs, n_vars))
                Gu_l[:, (self.N + 1) * self.dim_states + k * self.dim_inputs:
                          (self.N + 1) * self.dim_states + (k + 1) * self.dim_inputs] = -np.eye(self.dim_inputs)
                G_list.append(Gu_l)
                h_list.append(-np.array(self.env.input_lbs).reshape(-1, 1))
            if self.env.input_ubs is not None:
                Gu_u = np.zeros((self.dim_inputs, n_vars))
                Gu_u[:, (self.N + 1) * self.dim_states + k * self.dim_inputs:
                          (self.N + 1) * self.dim_states + (k + 1) * self.dim_inputs] = np.eye(self.dim_inputs)
                G_list.append(Gu_u)
                h_list.append(np.array(self.env.input_ubs).reshape(-1, 1))

        # Check if there are any constraints
        if len(G_list) > 0:
            G = np.vstack(G_list)
            h = np.vstack(h_list)
        else:
            G = np.zeros((1, n_vars))
            h = np.array([[1e10]])

        # Solve the QP
        solvers.options['show_progress'] = False
        sol = solvers.qp(matrix(H), matrix(f), matrix(G), matrix(h), matrix(Aeq), matrix(beq))
        z_opt = np.array(sol['x']).flatten()

        x_opt = z_opt[: (self.N + 1) * self.dim_states].reshape(self.N + 1, self.dim_states).T
        u_opt = z_opt[(self.N + 1) * self.dim_states:].reshape(self.N, self.dim_inputs).T

        if self.verbose:
            print(f"Optimal control action: {u_opt[:, 0]}")
            print(f"Predicted x: {x_opt}")
            print(f"Predicted u: {u_opt}")

        self.x_pred = x_opt
        self.u_pred = u_opt

        return u_opt[:, 0].flatten()+u_ref, x_opt.T, u_opt


<br>

#### **Task 2: Simulation and Visualization**  

1\) Specify the arguments and instantiate the controller class `LinearMPCController` as `LMPC`; 

- Parameters in the controller design:  

    i) weight for state $\bm{Q} = \text{diag}([1, 1])$ (requirement: symmetric, positive semi-definite matrix)  

    ii) weight for input $\bm{R} = [0.1]$ (requirement: symmetric, positive definite matrix)  
    
    iii) control frequency $f = 20$

    iv) horizon $N = 20$

2\) Instantiate the class `Simulator` and call function `run_simulation()` to generate the simulated state- and input-trajectory;

3\) Instantiate the class `Visualizer`, call function `display_final_results()` and `display_animation()` to show the simulations;

In [ ]:
# Define weight matrix in stage and terminal cost and the horizon for MPC 
Q = np.diag([1, 1])
R = np.array([[0.1]])
Qf = Q
N = 20

# Define parameters of simulation
freq = 20 # controll frequency
t_terminal = 8 # time length of simulation

# Define the LMPC controller
controller_mpc = LinearMPCController(env_constr, dynamics_constr, Q, R, Qf, freq, N, name='LMPC')

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc = Simulator(dynamics_constr, controller_mpc, env_constr, 1/freq, t_terminal)
simulator_mpc.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc = Visualizer(simulator_mpc)
visualizer_mpc.display_plots()
visualizer_mpc.display_animation()

<br>

#### **Task 3: Horizon and Terminal Components**  

In this section, we explore the effect of MPC hyperparameters on the overall control performance. Refer to the following instructions to run the simulations and analyze the results.

<blockquote style="padding-top: 20px; padding-bottom: 10px;">

##### **🔍 Hands-on Exploration: influence of the prediction horizon $N$ (primary)**

To gain an initial understanding of how the prediction horizon $N$ affects the behavior of the MPC controller, vary $N$ (ranging from $20$ to $160$) and repeat the simulations to observe the results.

Key aspects to pay attention to include:

- How similar are the MPC policy and the OCP policy?

- How does the computation time compare?

In [ ]:
# Define the prediction horizon in MPC
N = 20 # you may try different values from 20 to 160

In [ ]:
print(f"Weight matrix in terrmianl cost: {Qf}")

# Define the LMPC controller
controller_mpc = LinearMPCController(env_constr, dynamics_constr, Q, R, Qf, freq, N, name='MPC') 

start_time = time.time()

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc = Simulator(dynamics_constr, controller_mpc, env_constr, 1/freq, t_terminal)
simulator_mpc.run_simulation()

print(f"Average computation time per step: {(time.time()-start_time) / (freq * t_terminal)}")

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc = Visualizer(simulator_mpc)
visualizer_mpc.display_plots()

In [ ]:
visualizer_mpc.display_contrast_plots(simulator_ocp_2n)
visualizer_mpc.display_contrast_animation_same(simulator_ocp_2n)

#### **Results Analysis**

By comparing the simulation results of full-horizon open-loop OCP and MPC, we can draw the following conclusions:

- When the prediction horizon $N$ is relatively small:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\oplus$ **The computation time required for each MPC update is significantly shorter than that of solving the full OCP**. For example, when $N=20$, the average computation time is only $4.5$ ms, which makes online deployment much more feasible. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\ominus$ However, at the same time, the resulting **MPC policy is noticeably less aggressive compared to the full-horizon OCP solution**. This is because the MPC is "short-sighted" and only finds an optimal solution over the reduced horizon.

- As the prediction horizon $N$ increases, the MPC policy progressively approaches the full-horizozn OCP policy. When $N=160$, the two policies are nearly identical. 

<br>

**While a longer prediction horizon enables the controller to plan more aggressive and optimal trajectories, it also introduces a substantial computational burden.** This trade-off between control performance and real-time feasibility is a central consideration in practical MPC design.

Fortunately, this issue can be mitigated by introducing well-designed terminal components, such as a terminal cost. An appropriate terminal cost function accounts for the short-sightedness of the MPC and captures the impact of any future steps. For LMPC with a quadratic cost, a common approach is to **use the total cost of an infinite-horizon LQR as the terminal cost**.

Please run the example below and observe how this setting influences the closed-loop performance.

In [ ]:
# Define the prediction horizon in MPC
N = 20 # set the horizon to be a small value

# Linearize dynamics at equilibrium and solve DARE to compute the infinite LQR weight matrix
A_d, B_d = dynamics.get_linearized_AB_discrete(np.array([0.0, 0.0]), 0.0, 1/freq)
P = scipy.linalg.solve_discrete_are(A_d, B_d, Q, R)

print(f"Weight matrix in terrmianl cost: {P}")

# Define the LMPC controller
controller_mpc = LinearMPCController(env_constr, dynamics_constr, Q, R, P, freq, N, name='MPC') 

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc = Simulator(dynamics_constr, controller_mpc, env_constr, 1/freq, t_terminal)
simulator_mpc.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc = Visualizer(simulator_mpc)
visualizer_mpc.display_contrast_plots(simulator_ocp_2n)
visualizer_mpc.display_contrast_animation_same(simulator_ocp_2n)

#### **Results Analysis**

By incorporating the infinite-horizon LQR cost-to-go as the terminal cost in LMPC, the controller can achieve an aggressive and near-optimal policy, even with a relatively short prediction horizon $N$. **This demonstrates that a well-designed terminal cost can effectively compensate for a limited horizon, enabling a balanced trade-off between online computational tractability and policy optimality.**

The **key insight** behind this phenomenon lies in the structure of optimal control problems. When the prediction horizon is short, the controller does not plan for long-term consequences, often leading to suboptimal control inputs.
However, by embedding the LQR cost-to-go as the terminal cost, we provide the controller with an approximation of the infinite-horizon value function beyond the finite horizon.
This effectively extends the controller’s planning capability without increasing the computational burden, allowing it to make decisions that account for long-term benefits while operating within a short horizon.
In essence, the terminal cost serves as a surrogate for future cost.

<br>

#### **Task 4: LMPC vs. LQR**  

In this section, we compare linear MPC and LQR from Chapter 2. For LQR, explicit constraint handling is not supported. Therefore, we mimic the "clipping" behavior observed in physical systems — if the computed control input exceeds the allowed bounds, it is truncated to stay within the specified limits. In contrast, the MPC controller supports direct incorporation of constraints, allowing us to explicitly define input and state bounds during optimization.

More detailed, the case index and the constraints are:

- Test case: case 1 (flat ground)

- State constraints: $ \quad p \in [-2.0, 2.0], \quad v \in [-1.0, 1.0],$  

- Input constraints: $ \quad a \in [-0.25, 0.25],$  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Note that: in this example we deliberately use a tight input constraint to highlight the impact of trajectory saturation on the controller’s behavior.*

By introducing these constraints, we simulate the system's response and contrast the two controllers.

In [ ]:
# linear case number
case_linear = 1

# Define the physical boundary condition
input_lbs = -0.25
input_ubs = 0.25

# Define weight matrix in stage and terminal cost and the horizon for MPC 
Q = np.diag([1, 1])
R = np.array([[0.1]])
Qf = Q

In [ ]:
# Instantiate class 'Env'
env_linear = Env(case_linear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
# Instantiate class 'Dynamics'
dynamics_linear = Dynamics(env_linear)
# Instantiate the LQR controller class
controller_lqr_linear = LQRController(env_linear, dynamics_linear, Q, R, freq, name='LQR_case1_ideal')
controller_lqr_linear.set_lin_point(np.array([0.0, 0.0]))
# Instantiate the simulator, and then run the simulation
simulator_lqr_linear = Simulator(dynamics_linear, controller_lqr_linear, env, 1/freq, t_terminal)
simulator_lqr_linear.run_simulation()

# Instantiate class 'Env' and visualize the shape of the slope (left side) and theta curve (right side) 
env_linear_clipped = Env(case_linear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]), 
                           input_lbs=input_lbs, input_ubs=input_ubs)
# Instantiate the LQR controller class
controller_lqr_linear_clipped = LQRController(env_linear_clipped, dynamics_linear, Q, R, freq, name='LQR_case1_clipped')
controller_lqr_linear_clipped.set_lin_point(np.array([0.0, 0.0]))
# Instantiate the simulator, and then run the simulation
simulator_lqr_linear_clipped = Simulator(dynamics_linear, controller_lqr_linear_clipped, env_linear_clipped, 1/freq, t_terminal)
simulator_lqr_linear_clipped.run_simulation()
# Instantiate the visualizer, and display the plottings and animation
visualizer_lqr_linear_clipped = Visualizer(simulator_lqr_linear_clipped)
visualizer_lqr_linear_clipped.display_contrast_plots(simulator_lqr_linear)
visualizer_lqr_linear_clipped.display_contrast_animation(simulator_lqr_linear)

In [ ]:
N = 20

# Instantiate the MPC controller class (controller name must be different from the rest controllers)
controller_mpc_linear_constr = MPCController(env_linear_clipped, dynamics_linear, Q, R, Qf, freq, N, name="MPC_case1_constr")

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc_linear_constr = Simulator(dynamics_linear, controller_mpc_linear_constr, env_linear_clipped, 1/freq, t_terminal)
simulator_mpc_linear_constr.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc_linear_constr = Visualizer(simulator_mpc_linear_constr)
visualizer_mpc_linear_constr.display_plots()
visualizer_mpc_linear_constr.display_animation()


In [ ]:
# Show comparison between 2 cases
visualizer_mpc_linear_constr.display_contrast_plots(simulator_lqr_linear_clipped)
visualizer_mpc_linear_constr.display_contrast_animation_same(simulator_lqr_linear_clipped)

#### **Results Analysis:**

Simulation results indicate that LQR, due to its inability to explicitly enforce input and state constraints, may generate control policies that exceed these bounds during execution. In such cases, actuator-level clipping leads to saturation, causing the actual control trajectory to deviate from the ideal one. Nevertheless, thanks to the global convergence property of linear systems, the controller still manages to reach the target state, albeit with noticeable delays.

In contrast, LMPC is able to explicitly handle both input and state constraints within the optimization process, thereby avoiding such trajectory saturations and ensuring constraint-compliant tracking performance.

<br>


<br>

----

### **Part (c): Nonlinear MPC**

In this section, we will provide the implementation of nonlinear MPC (NMPC) and demonstrate how it can be applied to a stabilization task. We will also visualize the performance of the implemented NMPC.


##### **NMPC Problem Formulation:**

$$
J_k^*(\boldsymbol{x_k}) = \min_{u_{k|k}, \ldots, u_{k+N-1|k}} 
g_N\left( \boldsymbol{x_{k+N|k}} \right) + \sum_{i=0}^{N-1} g_i\left( \boldsymbol{x_{k+i|k}}, \boldsymbol{u_{k+i|k}} \right)
$$

subject to:

$$
\boldsymbol{x_{k+i+1|k}} = \boldsymbol{f} \left( \boldsymbol{x_{k+i|k}}, \boldsymbol{u_{k+i|k}} \right), \quad \forall i \in \{0, \ldots, N-1\},
$$

$$
\boldsymbol{x_{k+i|k}} \in \mathcal{X}, \quad \forall i \in \{0, \ldots, N\},
$$

$$
u_{k+i|k} \in \mathcal{U}, \quad \forall i \in \{0, \ldots, N-1\},
$$

$$
\boldsymbol{x_{k+N|k}} \in \mathcal{X}_f, 
$$

$$
\boldsymbol{x_{k|k}} = \boldsymbol{\overline{x}_{k}}
$$

##### **Cost design in stabilization task:**

- Stage cost: The cost function consists of a tracking error term and an input penalty term. The tracking error term minimizes the deviation between the state trajectory and the target state, while the input penalty term prevents the controller from applying excessively large.

$$
g_i\left( \boldsymbol{x_{k+i|k}}, \boldsymbol{u_{k+i|k}} \right) = \left( \boldsymbol{x_{k+i|k}} - \boldsymbol{x_T} \right)^T \boldsymbol{Q} \left( \boldsymbol{x_{k+i|k}} - \boldsymbol{x_T} \right) + u_{k+i|k} ^T \boldsymbol{R} u_{k+i|k}
$$

- Terminal cost: only the tracking error term

$$
g_N\left( \boldsymbol{x_{k+N|k}} \right) = \left( \boldsymbol{x_{k+N|k}} - \boldsymbol{x_T} \right)^T \boldsymbol{Q_f} \left( \boldsymbol{x_{k+N|k}} - \boldsymbol{x_T} \right)
$$


##### **Reformulation of the Original NLP Problem:**

For nonlinear programming (NLP) problems that employ a quadratic cost function, a common approach is to reformulate the problem as a quadratic cost optimization problem with nonlinear constraints to leverage specialized solvers. In this reformulation, the state and control variables at each shooting node are combined into a single augmented variable, and the cost function is rewritten as a quadratic function of this new augmented variable. The reformulated states and costs can be expressed as follows:

- Intermediate shooting node (related to stage cost term $\forall i \in \{0, \ldots, N-1\}$): 

$$
\boldsymbol{z_{k+i|k}} = [\boldsymbol{x_{k+i|k}}^T, \boldsymbol{u_{k+i|k}}^T]^T,
$$

$$
\boldsymbol{z}_{ref} = [\boldsymbol{x_T}^T, \boldsymbol{0}^{1 \times m}]^T,
$$

$$
g_i\left( \boldsymbol{x_{k+i|k}}, \boldsymbol{u_{k+i|k}} \right) = \left( \boldsymbol{z_{k+i|k}} - \boldsymbol{z}_{ref} \right)^T \begin{bmatrix} \boldsymbol{Q} & \boldsymbol{0}^{n \times m} \\ \boldsymbol{0}^{m \times n} & \boldsymbol{R} \end{bmatrix} \left( \boldsymbol{z_{k+i|k}} - \boldsymbol{z}_{ref} \right)
$$

- Terminal shooting node (related to terminal cost term $i = N$):

$$
\boldsymbol{z_{k+N|k}} = \boldsymbol{x_{k+N|k}},
$$

$$
\boldsymbol{z}_{f,ref} = \boldsymbol{x_T}, 
$$

$$
g_N\left( \boldsymbol{x_{k+N|k}} \right) = \left( \boldsymbol{z_{k+N|k}} - \boldsymbol{z}_{f,ref} \right)^T \boldsymbol{Q_f} \left( \boldsymbol{z_{k+N|k}} - \boldsymbol{z}_{f,ref} \right)
$$

##### **Solving the NMPC Problem:**

In the linear MPC section, we showed how the optimal control problem (OCP) can be reformulated into a lifted form as a QP, where all decision variables are stacked together, and the dynamics are expressed as linear equality constraints.

However, in nonlinear MPC, the system dynamics are nonlinear:

$$
\boldsymbol{x_{k+1}} = \boldsymbol{f} \left( \boldsymbol{x_{k}}, \boldsymbol{u_{k}} \right)
$$

This leads to a nonlinear programming (NLP) problem once we stack all decision variables. To solve such problems, we need to resort to **Sequential Quadratic Programming (SQP)**, which we introduced in the **Optimization Fundamentals** chapter. SQP iteratively approximates the nonlinear problem using a sequence of QP subproblems that are solved at each step.


##### **NMPC Solver:**

Rewriting and solving nonlinear OCPs manually is often **cumbersome and error-prone**. To streamline implementation, we can use specialized optimal control toolkits that:

- Automatically handle dynamics, cost, and constraint modeling 

- Generate efficient code for fast real-time solutiond;

Some widely used NMPC interfaces and solvers include:

- **Acados (Interface)**: fast, tailored for OCP problems, supports real-time code generation;

- **CasADi (Interface) + IPOPT (Solver)**: flexible, symbolic modeling;

- Solvers: **FORCES Pro**, **HPIPM**, and others;

In this section, we use **Acados** for our implementation since it is tailored to optimal control problems and is, therefore, particularly computationally efficient.

<br>

#### **Task 1: Solver Configuration**  

In this step, we will implement the setup function `setup_external()`, which will be automatically called in the constructor of class `MPCController`. We use Acados as the solver for our MPC problem, which offers a rich Python interface for defining and solving optimization problems. The Python interface of Acados allows users to describe the dynamics, cost, and constraints symbolically. It then handles code generation, compilation, and solver setup internally, enabling fast and efficient online optimization. The implementation will follow three key steps:

1\) Instantiate the class `AcadosModel`, specify the state / input vector and the symbolic system dynamcis;  
    Hint: you may get the state / input vectors and the system dynamcis from class `Dynamics` 

2\) Instantiate the class `AcadosOcp`, specify the model and the optimization problem (incl. solver configurations, costs, constraints, etc.);  
    Hint 1: for the standard formulation of an optimal control problem in Acados please refer to the attachment [problem_formulation_ocp_acados.pdf](./problem_formulation_ocp_acados.pdf)    
    Hint 2: the python interfaces provided by Acados for solver configuration please refer to this [website](https://docs.acados.org/python_interface/index.html)  


3\) Instantiate the class `AcadosOcpSolver` with the object of class `AcadosOcp`; 

In [ ]:
def setup_external(self) -> None:

    """
    Define the MPC optimization problem using Acados.
    """
    
    ## Model
    # Set up Acados model
    model = AcadosModel()
    model.name = self.name

    # Define model: x_dot = f(x, u)
    model.x = self.dynamics.states
    model.u = self.dynamics.inputs
    model.f_expl_expr = ca.vertcat(self.dynamics.dynamics_function(self.dynamics.states, self.dynamics.inputs))
    model.f_impl_expr = None # no needed, we already have the explicit model


    ## Optimal control problem
    # Set up Acados OCP
    ocp = AcadosOcp()
    ocp.model = model # link to the model (class: AcadosModel)
    ocp.dims.N = self.N  # prediction horizon
    ocp.solver_options.tf = self.N * self.dt  # total prediction time
    ocp.solver_options.qp_solver = "FULL_CONDENSING_HPIPM" # Partially condensing interior-point method
    ocp.solver_options.integrator_type = "ERK" # explicit Runge-Kutta
    ocp.solver_options.nlp_solver_type = "SQP" # sequential quadratic programming

    # Set up other hyperparameters in SQP solving
    ocp.solver_options.nlp_solver_max_iter = 100
    ocp.solver_options.nlp_solver_tol_stat = 1E-6
    ocp.solver_options.nlp_solver_tol_eq = 1E-6
    ocp.solver_options.nlp_solver_tol_ineq = 1E-6
    ocp.solver_options.nlp_solver_tol_comp = 1E-6
    
    # For debugging
    #ocp.solver_options.print_level = 2

    # Set up cost function
    ocp.cost.cost_type = "LINEAR_LS"
    ocp.cost.cost_type_e = "LINEAR_LS"
    ocp.cost.W = np.block([
        [self.Q, np.zeros((self.dim_states, self.dim_inputs))],
        [np.zeros((self.dim_inputs, self.dim_states)), self.R],
    ])
    ocp.cost.W_e = self.Qf

    # Set up mapping from QP to OCP
    # Define output matrix for non-terminal state
    ocp.cost.Vx = np.block([
        [np.eye(self.dim_states)],
        [np.zeros((self.dim_inputs, self.dim_states))]
    ])
    # Define breakthrough matrix for non-terminal state
    ocp.cost.Vu = np.block([
        [np.zeros((self.dim_states, self.dim_inputs))],
        [np.eye(self.dim_inputs)]
    ])
    # Define output matrix for terminal state
    ocp.cost.Vx_e = np.eye(self.dim_states)

    # Initialize reference of task (stabilization)
    ocp.cost.yref = np.zeros(self.dim_states + self.dim_inputs) 
    ocp.cost.yref_e = np.zeros(self.dim_states) 

    # Define constraints
    ocp.constraints.x0 = self.init_state  # Initial state

    # State constraints
    ocp.constraints.idxbx = np.arange(self.dim_states)
    ocp.constraints.idxbx_e = np.arange(self.dim_states)
    if self.env.state_lbs is None and self.env.state_ubs is None:
        ocp.constraints.lbx_0 = np.full(self.dim_states, -1e6)
        ocp.constraints.ubx_0 = np.full(self.dim_states, 1e6)
        ocp.constraints.lbx = np.full(self.dim_states, -1e6)
        ocp.constraints.ubx = np.full(self.dim_states, 1e6)
        ocp.constraints.lbx_e = np.full(self.dim_states, -1e6)
        ocp.constraints.ubx_e = np.full(self.dim_states, 1e6)
    elif self.env.state_lbs is not None and self.env.state_ubs is None:
        ocp.constraints.lbx_0 = np.array(self.env.state_lbs)
        ocp.constraints.ubx_0 = np.full(self.dim_states, 1e6)
        ocp.constraints.lbx = np.array(self.env.state_lbs)
        ocp.constraints.ubx = np.full(self.dim_states, 1e6)
        ocp.constraints.lbx_e = np.array(self.env.state_lbs)
        ocp.constraints.ubx_e = np.full(self.dim_states, 1e6)
    elif self.env.state_lbs is None and self.env.state_ubs is not None:
        ocp.constraints.lbx_0 = np.full(self.dim_states, -1e6)
        ocp.constraints.ubx_0 = np.array(self.env.state_ubs)
        ocp.constraints.lbx = np.full(self.dim_states, -1e6)
        ocp.constraints.ubx = np.array(self.env.state_ubs)
        ocp.constraints.lbx_e = np.full(self.dim_states, -1e6)
        ocp.constraints.ubx_e = np.array(self.env.state_ubs)
    else:
        ocp.constraints.lbx_0 = np.array(self.env.state_lbs)
        ocp.constraints.ubx_0 = np.array(self.env.state_ubs)
        ocp.constraints.lbx = np.array(self.env.state_lbs)
        ocp.constraints.ubx = np.array(self.env.state_ubs)
        ocp.constraints.lbx_e = np.array(self.env.state_lbs)
        ocp.constraints.ubx_e = np.array(self.env.state_ubs)
    
    # Input constraints
    ocp.constraints.idxbu = np.arange(self.dim_inputs)
    if self.env.input_lbs is None and self.env.input_ubs is None:
        ocp.constraints.lbu = np.full(self.dim_inputs, -1e6)
        ocp.constraints.ubu = np.full(self.dim_inputs, 1e6)
    elif self.env.input_lbs is not None and self.env.input_ubs is None:
        ocp.constraints.lbu = np.array(self.env.input_lbs)
        ocp.constraints.ubu = np.full(self.dim_inputs, 1e6)
    elif self.env.input_lbs is None and self.env.input_ubs is not None:
        ocp.constraints.lbu = np.full(self.dim_inputs, -1e6)
        ocp.constraints.ubu = np.array(self.env.input_ubs)
    else:
        ocp.constraints.lbu = np.array(self.env.input_lbs)
        ocp.constraints.ubu = np.array(self.env.input_ubs)


    ## Ocp Solver
    # Set up Acados solver
    self.ocp = ocp
    self.solver = AcadosOcpSolver(ocp, json_file=f"{model.name}.json")

    if self.verbose:
        print("MPC setup with Acados completed.")


<br>

#### **Task 2: Control Loop Definition**  

After defining the optimization problem using Acados' Python interface and generating the corresponding solver package, we need to assign values to variables and compute the control output in each control cycle. This process will be encapsulated in a control loop function `compute_action_external()`, which is called at every time step. In each cycle, you need to:

1\) Assign values for the initial state and the tracking reference via the interface `set()`;  

2\) Solve OCP problem by calling `solve()`;  

3\) Get the first input command through `get()`;  

In [ ]:
def compute_action_external(self, current_state: np.ndarray, current_time) -> np.ndarray:
    """
    Solve the MPC problem and compute the optimal control action.

    Args:
    - current_state: The current state of the system.
    - current_time: The current time (not used in this time-invariant case).

    Returns:
    - Optimal control action.
    """

    print(f"Current state: {current_state}")
    print(f"Target state: {self.target_state}")


    # Update initial state in the solver
    self.solver.set(0, "lbx", current_state)
    self.solver.set(0, "ubx", current_state)

    # Update reference trajectory for all prediction steps
    state_ref = self.target_state
    input_ref = np.zeros(self.dim_inputs)
    for i in range(self.N):
        self.solver.set(i, "yref", np.concatenate((state_ref, input_ref)))
    self.solver.set(self.N, "yref", state_ref) # set reference valur for y_N seperately (different shape)

    # Solve the MPC problem
    status = self.solver.solve()
    #if status != 0:
    #    raise ValueError(f"Acados solver failed with status {status}")

    # Extract the first control action
    u_optimal = self.solver.get(0, "u")

    # Extract the predictions
    x_pred = np.zeros((self.N + 1, self.dim_states))
    u_pred = np.zeros((self.N, self.dim_inputs))
    for i in range(self.N + 1):
        x_pred[i, :] = self.solver.get(i, "x")
        if i < self.N:
            u_pred[i, :] = self.solver.get(i, "u")

    if self.verbose:
        print(f"Optimal control action: {u_optimal}")
        #print(f"x_pred: {x_pred}")
        #print(f"u_pred: {u_pred}")

    return u_optimal, x_pred, u_pred

<br>

#### **Task 3: Simulation and Visualization**  

1\) Bind the defined setup function `setup_external()` and control loop function `compute_action_external()` to class `MPCController`;

2\) Specify the arguments and instantiate the controller class `MPCController` as `MPC_0`; 

- Parameters in the controller design:  

    i) weight for state $\bm{Q} = \text{diag}([1, 1])$ (requirement: symmetric, positive semi-definite matrix)  

    ii) weight for input $\bm{R} = [0.1]$ (requirement: symmetric, positive definite matrix)  
    
    iii) control frequency $f = 20$

    iv) horizon $N = 20$

3\) Instantiate the class `Simulator` and call function `run_simulation()` to generate the simulated state- and input-trajectory;

4\) Instantiate the class `Visualizer`, call function `display_final_results()` and `display_animation()` to show the simulations;


In [ ]:
# Bind the defined MPC controller setup function to the corresponding class, will be automatically called by constructor
MPCController.setup = setup_external
MPCController.compute_action = compute_action_external

# Define weight matrix in stage and terminal cost and the horizon for MPC 
Q = np.diag([1, 1])
R = np.array([[0.1]])
Qf = Q
N = 20

# Define parameters of simulation
freq = 20 # controll frequency
t_terminal = 8 # time length of simulation

# Instantiate the MPC controller class
# Arguments: 
#   1) an object of class `Env` (to deliver infos about initial state, constraints, etc.), type: Env  
#   2) an object of class `Dynamics` (to deliver infos about symbolic system dynamics), type: Dynamics  
#   3) weight matrices in cost functions: i) `Q`: weight metrix of state, type: np.array  
#                                         ii) `R`: weight metrix of input, type: np.array  
#                                         iii) `Q_f`: weight metrix of state in terminal cost, type: np.array  
#   4) freq: control frequency $f$ , type: int  
#   5) N: horizon in MPC , type: int  
#   6) name: the name of current controller displayed in plots, type: string
controller_mpc = MPCController(env_constr, dynamics_constr, Q, R, Qf, freq, N, name="MPC_1")

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc = Simulator(dynamics_constr, controller_mpc, env_constr, 1/freq, t_terminal)
simulator_mpc.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc = Visualizer(simulator_mpc)
visualizer_mpc.display_plots()
visualizer_mpc.display_animation()

<br>

#### **Task 4: NMPC vs. ILQR**  

In the following, we compare the performance of NMPC (Model Predictive Control for nonlinear systems) with the ILQR implementation introduced in Chapter 4. To this end, we introduce the following set of constraints:

- Test case: case 4 (climbing up the hill)

- State constraints: $ \quad p \in [-2.0, 2.0], \quad v \in [-4.0, 4.0],$  

- Input constraints: $ \quad a \in [-8.0, 8.0],$  

By introducing these constraints, we simulate the system's response and contrast the controllers.

In [ ]:
# nonlinear case number
case_nonlinear = 4

# Define the physical boundary condition
state_lbs = np.array([-2.0, -4.0])
state_ubs = np.array([2.0, 4.0])
input_lbs = -8.0
input_ubs = 8.0

# Define weight matrix in stage and terminal cost and the horizon for MPC 
Q = np.diag([10, 10])
R = np.array([[0.1]])
Qf = Q

In [ ]:
# Instantiate class 'Env'
env_nonlinear = Env(case_nonlinear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
# Instantiate class 'Dynamics'
dynamics_nonlinear = Dynamics(env_nonlinear)
# Instantiate the LQR controller class
controller_lqr_nonlinear = LQRController(env_nonlinear, dynamics_nonlinear, Q, R, freq, name='LQR_case4_ideal', verbose=None)
controller_lqr_nonlinear.set_lin_point(np.array([0.0, 0.0]))
# Instantiate the simulator, and then run the simulation
simulator_lqr_nonlinear = Simulator(dynamics_nonlinear, controller_lqr_nonlinear, env_nonlinear, 1/freq, t_terminal)
simulator_lqr_nonlinear.run_simulation()
# Use trajectories getting from LQR to initialize ILQR
_, input_traj_lqr_nonlinear = simulator_lqr_nonlinear.get_trajectories()

# Instantiate the ILQR controller class
controller_ilqr_nonlinear = iLQRController(env_nonlinear, dynamics_nonlinear, Q, R, Qf, freq, name='ILQR_case4_ideal', verbose=None)
# Initialize ILQR controller with trajectory from LQR, then run setup function to compute optimal policy
controller_ilqr_nonlinear.setup(input_traj_lqr_nonlinear)
# Instantiate the simulator, run the simulation, and plot the results
simulator_ilqr_nonlinear = Simulator(dynamics_nonlinear, controller_ilqr_nonlinear, env_nonlinear, 1/freq, t_terminal)
simulator_ilqr_nonlinear.run_simulation()

# Instantiate class 'Env' and visualize the shape of the slope (left side) and theta curve (right side) 
env_nonlinear_clipped = Env(case_nonlinear, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]), 
                           input_lbs=input_lbs, input_ubs=input_ubs)
# Instantiate the ILQR controller class
controller_ilqr_nonlinear_clipped = iLQRController(env_nonlinear_clipped, dynamics_nonlinear, Q, R, Qf, freq, name='ILQR_case4_clipped')
# Initialize ILQR controller with trajectory from LQR, then run setup function to compute optimal policy
controller_ilqr_nonlinear_clipped.setup(input_traj_lqr_nonlinear)
# Instantiate the simulator, and then run the simulation
simulator_ilqr_nonlinear_clipped = Simulator(dynamics_nonlinear, controller_ilqr_nonlinear_clipped, env_nonlinear_clipped, 1/freq, t_terminal)
simulator_ilqr_nonlinear_clipped.run_simulation()
# Instantiate the visualizer, and display the plottings and animation
visualizer_ilqr_nonlinear_clipped = Visualizer(simulator_ilqr_nonlinear_clipped)
visualizer_ilqr_nonlinear_clipped.display_contrast_plots(simulator_ilqr_nonlinear)
visualizer_ilqr_nonlinear_clipped.display_contrast_animation_same(simulator_lqr_nonlinear)

In [ ]:
N = 60

# Instantiate the MPC controller class (controller name must be different from the rest controllers)
controller_mpc_nonlinear_constr = MPCController(env_nonlinear_clipped, dynamics_nonlinear, Q, R, Qf, freq, N, name="MPC_case4_constr")

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc_nonlinear_constr = Simulator(dynamics_nonlinear, controller_mpc_nonlinear_constr, env_nonlinear_clipped, 1/freq, t_terminal)
simulator_mpc_nonlinear_constr.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc_nonlinear_constr = Visualizer(simulator_mpc_nonlinear_constr)
visualizer_mpc_nonlinear_constr.display_plots()
visualizer_mpc_nonlinear_constr.display_animation()

In [ ]:
# Show comparison between 2 cases
visualizer_mpc_nonlinear_constr.display_contrast_plots(simulator_ilqr_nonlinear_clipped)
visualizer_mpc_nonlinear_constr.display_contrast_animation_same(simulator_ilqr_nonlinear_clipped)

#### **Results Analysis:**

The simulation results reveal that, in this case, the ILQR-controlled system fails to be stabilized around the target state. This failure arises from the fact that equilibrium points in nonlinear systems generally do not exhibit global convergence. When actuator saturation occurs — typically due to clipping beyond the input bounds — the system trajectory may become trapped in an unstable region, rendering the controller ineffective.

In contrast, NMPC explicitly incorporates input and state constraints into the optimization process, thereby avoiding unexpected saturation. Although the NMPC controller is subject to the same input constraints, it is able to leverage trajectory prediction over a finite horizon to plan ahead. Instead of attempting to reach the top in a single move—which is infeasible due to the limited input—it intentionally generates a control sequence that builds up momentum over time. This can be observed from the fact that the cart reaches a higher velocity when passing the starting point for the second time. As a result, despite the input limitations, the cart successfully climbs to the top after two rounds of back-and-forth motion, demonstrating the planner’s ability to exploit system dynamics over time.

<br>


----

### **Part (d): Hyperparameter Selection**

The main hyperparameters in MPC include the prediction horizon $N$, sampling frequency $f$, and weighting matrices $\boldsymbol{Q}$, $\boldsymbol{R}$, and $\boldsymbol{Q}_f$, etc. The influence of these hyperparameters are as follows:

  1) Prediction horizon $N$: A shorter prediction horizon reduces computation time per cycle, improving real-time compatibility, but may result in suboptimal or overly reactive behavior due to the limited foresight. A longer horizon allows the controller to anticipate future states more effectively, often producing smoother or more optimal trajectories, but at the cost of increased computational load and longer computation time per cycle;

  2) Sampling frequency $f$: A higher sampling frequency (i.e., smaller sampling time) enables faster control updates and can predict the behavior of the high-frequency system dynamics more accurately, leading to better control performance, especially in fast systems. However, it increases the computational burden due to more frequent updates. Conversely, a lower sampling frequency reduces computational demand but may degrade performance or even destabilize the system if the controller cannot react sufficiently fast;

  3) Weighting matrices $\boldsymbol{Q}$, $\boldsymbol{R}$, and $\boldsymbol{Q}_f$: These matrices define the relative importance of state tracking versus control effort in the cost function. $\boldsymbol{Q}$ penalizes deviation from desired states, $\boldsymbol{R}$ penalizes excessive or aggressive control inputs, and $\boldsymbol{Q}_f$ defines the terminal state penalty, encouraging the system to reach the final desired state smoothly. As in LQR, tuning these matrices allows you to balance tracking accuracy and control smoothness or energy efficiency.

##### **Example: Prediction Horizon**

As discussed in Chapter 2, the impact of the weight matrices on the performance of MPC has been well-established. The prediction horizon $N$, as another important and unique hyperparameter in the MPC algorithm, similarly plays a crucial role. Therefore, in this section, we will focus on exploring the impact of the horizon length on the performance of the MPC algorithm.

To better highlight the impact of the prediction horizon, we specifically designed the following task setup:

- Environment: case 4, varying slope, underactuated case (shape as shown in the output of the first block)

- Initial/target state: $ \quad \boldsymbol{x}_s = [-0.5, 0]^T, \quad \boldsymbol{x}_T = [0.6, 0]^T,$  

- State constraints: $ \quad p \in [-2.0, 2.0], \quad v \in [-4.0, 4.0],$  

- Input constraints: $ \quad a \in [-5.0, 5.0],$  

The hyperparameters are:

- Weight for state $\bm{Q} = \text{diag}([1, 1])$ (requirement: symmetric, positive semi-definite matrix)  

- Weight for input $\bm{R} = [0.1]$ (requirement: symmetric, positive definite matrix)  
    
- Control frequency $f = 20$

- Horizon $N = 20 / 60 / 100$ **(Control Group)**

Based on the above task and controller setup, we instantiate various configurations and perform simulations. Vary the prediction horizon using values of 20, 60, and 100, respectively, and observe the corresponding simulation results.

In [ ]:
# Define the case index for the slope profile
case = 4 

# Define the initial and target state
initial_position = -0.5
initial_velocity = 0.0
target_position = 0.6
target_velocity = 0.0

# Define the physical boundary condition
state_lbs = np.array([-2.0, -4.0])
state_ubs = np.array([2.0, 4.0])
input_lbs = -5.0
input_ubs = 5.0

# Instantiate class 'Env' and visualize the shape of the slope (left side) and theta curve (right side) 
#env = Env(case, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]))
env = Env(case, np.array([initial_position, initial_velocity]), np.array([target_position, target_velocity]),
          state_lbs=state_lbs, state_ubs=state_ubs, input_lbs=input_lbs, input_ubs=input_ubs)
env.test_env()

# Instantiate class 'Dynamics'
dynamics = Dynamics(env)

In [ ]:
# Define weight matrix in stage and terminal cost
Q = np.diag([1, 1])
R = np.array([[0.1]])
Qf = Q

# Define parameters of simulation
freq = 20 # controll frequency
t_terminal = 6 # time length of simulation

In [ ]:
N = 20

# Define parameters of simulation
freq = 20 # controll frequency
t_terminal = 6 # time length of simulation

# Instantiate the MPC controller class (controller name must be different from the rest controllers)
controller_mpc_N20 = MPCController(env, dynamics, Q, R, Qf, freq, N, name="MPC_N20")

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc_N20 = Simulator(dynamics, controller_mpc_N20, env, 1/freq, t_terminal)
simulator_mpc_N20.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc_N20 = Visualizer(simulator_mpc_N20)
visualizer_mpc_N20.display_plots(show_text='N = 20')
visualizer_mpc_N20.display_animation()

In [ ]:
N = 60

# Instantiate the MPC controller class (controller name must be different from the rest controllers)
controller_mpc_N60 = MPCController(env, dynamics, Q, R, Qf, freq, N, name="MPC_N60")

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc_N60 = Simulator(dynamics, controller_mpc_N60, env, 1/freq, t_terminal)
simulator_mpc_N60.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc_N60 = Visualizer(simulator_mpc_N60)
visualizer_mpc_N60.display_plots(show_text='N = 60')
visualizer_mpc_N60.display_animation()

In [ ]:
N = 100

# Instantiate the MPC controller class (controller name must be different from the rest controllers)
controller_mpc_N100 = MPCController(env, dynamics, Q, R, Qf, freq, N, name="MPC_N100")

# Instantiate the simulator, run the simulation, and plot the results
simulator_mpc_N100 = Simulator(dynamics, controller_mpc_N100, env, 1/freq, t_terminal)
simulator_mpc_N100.run_simulation()

# Instantiate the visualizer, and display the plottings and animation
visualizer_mpc_N100 = Visualizer(simulator_mpc_N100)
visualizer_mpc_N100.display_plots(show_text='N = 100')
visualizer_mpc_N100.display_animation()

In [ ]:
# Show comparison between all cases
visualizer_mpc_N20.display_contrast_plots(simulator_mpc_N60, simulator_mpc_N100)
visualizer_mpc_N20.display_contrast_animation_same(simulator_mpc_N60, simulator_mpc_N100)

##### **Results Analysis: System Response Under Different Horizon Lengths**

For the defined task, while keeping the value of all other hyperparameters fixed, we vary the value of $N$ to represent short, medium, and long horizon cases. From the experimental results, the following trends can be observed:

1. **Short prediction horizon** ($N = 20$):  
   - Under this condition, the car is unable to reach the target position. Instead, after a brief oscillation, it comes to rest at the bottom of the valley;  
   - This outcome arises because, within the current prediction horizon (20 steps at 20 Hz, i.e., 1 second), the presence of input constraints makes it impossible to reach the goal even under a bang-bang control strategy. Consequently, the MPC prioritizes minimizing the penalty term on input, leading to a zero-input solution.

2. **Medium prediction horizon** ($N = 60$):  
   - In this scenario, the car begins by moving backward, undergoes cycles of back-and-forth motion with increasing amplitude, and then propels itself toward the top, successfully reaching the target location;
   - This behavior arises because, in the underactuated case, the car is unable to generate sufficient input to reach the top in a single ascent. However, when the prediction horizon is sufficiently long, the system can perform periodic back-and-forth motions to accumulate additional momentum. Once enough kinetic energy has been gained, the car is able to propel itself to the top.

3. **Long prediction horizon** ($N = 100$):  
   - In this case, compared to the medium prediction horizon scenario, the car requires fewer cycles to accumulate enough momentum accumulation to reach the top;
   - This is because, compared to the case with N=60, the prediction horizon with N=100 covers a longer time interval. Specifically, N=60 corresponds to a prediction window of 3s (60 steps at 20 Hz), while the transient behavior in this case lasts approximately 4 seconds, as observed from the simulation results. As a result, at the start of the optimization, the MPC cannot foresee the car reaching the top, and thus chooses a strategy involving more cycles of momentum accumulation. 
   - On the other hand, with N=100, the prediction horizon spans 5 seconds. Therefore, the prediction horizon fully encompasses the transient behavior, allowing the MPC to effectively optimize over approximately the whole timeline. As a result, the computed control sequence can be regarded as a near-optimal policy.

##### **Main Conclusion:**

In summary, the length of the prediction horizon significantly influences the system's ability to generate effective control strategies. A short horizon limits the controller's foresight, leading to suboptimal or even ineffective behavior. In contrast, a sufficiently long horizon allows the MPC to anticipate the full transient dynamics and compute near-optimal control actions. Therefore, selecting an appropriate horizon length is critical for achieving desired performance in MPC design.

